In [ ]:
# Start writing code here...

# Wisoo

%run "./calibrateCamera.ipynb"

     |████████████████████████████████| 7.0 MB 28.8 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 49.5 MB 266 kB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
./calibrateCamera.ipynb:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
./calibrateCamera.ipynb:93: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Input:   
Camera calibration: calMatrix and distCoeffs.  
(outputs from calibrateCamera().   
Image of the two AprilBoards setup with object to be scanned.   
ES 143 AprilBoard datafile (AprilBoards.pickle).   
Output:  
plane_h, plane_v -- Two homogeneous 4-vectors corresponding to planes (\Pi_v, \Pi_h) in the camera-centered coordinate system.  

In [ ]:
!pip install av
!pip install pims
!pip install pillow
!pip install plotly
import av
import pims
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go


     |████████████████████████████████| 36.9 MB 6.1 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 85 kB 6.8 MB/s 
  Created wheel for pims: filename=PIMS-0.5-py3-none-any.whl size=84326 sha256=1e6477b8dc6581740308e4089971501b205a8f8cbed720fe145b67b2e1bdfda7
  Stored in directory: /home/jovyan/.cache/pip/wheels/75/02/a9/86571c38081ba4c1832eb95430b5d588dfa15a738e2a603737
Successfully built pims
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 13.1 MB 15.5 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=bce96651195c0783272c293f917efc4d9d26d86828fd20c5578b276f65b406ba
  Stored in directory: /home/jovyan/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287

In [ ]:
## Use the following code for indexing image frame in the video.
## Needed it because the calib0**.png images didnt have two boards in the images
# v = pims.Video('./resources_unzipped/shadow.MOV')

# for num, i in enumerate(v[:20]):
#     im = Image.fromarray(v[num])
#     im.save(f'./resources_unzipped/two_boards/frame{num}.png')
    



In [ ]:
v = pims.Video('./resources_unzipped/shadow2.MOV')
test_img = v[5]
print(test_img.shape)

(1080, 1920, 3)


In [ ]:
def detect_aprilboard_v2(img, board, apriltag_detector):
    # Usage:  imgpoints, objpoints, tag_ids = detect_aprilboard(img,board,AT_detector)
    #
    # Input: 
    #   image -- grayscale image
    #   board -- at_coarseboard or at_fineboard (list of dictionaries)
    #   AT_detector -- AprilTag Detector parameters
    #
    # Returns: 
    #   imgpoints -- Nx2 numpy array of (x,y) image coords
    #   objpoints -- Nx3 numpy array of (X,Y,Z=0) board coordinates (in inches)
    #   tag_ids -- Nx1 list of tag IDs
    
    imgpoints=[]
    objpoints=[]
    tagIDs=[]
    
    # detect april tags
    imgtags = apriltag_detector.detect(img, 
                                    estimate_tag_pose=False, 
                                    camera_params=None, 
                                    tag_size=None)

    if len(imgtags):
        # collect image coordinates of tag centers
        # imgpoints = np.vstack([ sub.center for sub in tags ])

        # list of all tag_id's that are in board
        brdtagIDs = [ sub['tag_id'] for sub in board ]

        # list of all detected tag_id's that are in image
        imgtagIDs = [ sub.tag_id for sub in imgtags ]

        # list of all tag_id's that are in both
        tagIDs = list(set(brdtagIDs).intersection(imgtagIDs))
        
        if len(tagIDs):
            # all board list-elements that contain one of the common tag_ids
            objs=list(filter(lambda tagnum: tagnum['tag_id'] in tagIDs, board))
            
            # their centers
            objpoints = np.vstack([ sub['center'] for sub in objs ])
    
            # all image list-elements that contain one of the detected tag_ids
            imgs=list(filter(lambda tagnum: tagnum.tag_id in tagIDs, imgtags))    
            
            # their centers
            imgpoints = np.vstack([ sub.center for sub in imgs ])
        
    return imgpoints, objpoints, tagIDs

In [ ]:
def add_camera(h,w,camera,raysize,figobj):
# Add tetrahedral camera to pyplot figure
# h,w:      height and width of image in pixels
# camera:   3x4 camera matrix
# raysize:  length of tetrahedral edges (in world units)
# fig:      pyplot figure object
#
# Returns: 1
#
# Uses anatomy of camera matrices from Hartley and Zisserman Chapter 6

    # normalize camera such that bottom-left three-vector 
    #   corresponds to unit-length principal ray in front of camera (HZ Section 6.2.3)
    camera=camera*np.sign(np.linalg.det(camera[:,0:3]))/np.linalg.norm(camera[2,0:3])
    
    # Compute camera center (null vector of P), because Camera center is where every ray meets 
    _, _, v = np.linalg.svd(camera)
    C = np.transpose(v[-1,0:3]) / v[-1,3]

    # Back-project image corners to unit-length 3D ray segments:
    S = np.array([[0, 0, 1],       # homog image coords if top left pixel
                  [0, h-1, 1],     # bottom left 
                  [w-1, h-1, 1],   # bottom right
                  [w, 0, 1]])      # top right
    
    #   HZ equation (6.14): compute one 3D point along each ray
    X = np.transpose(np.linalg.lstsq(
        camera[:,0:3], 
        np.transpose(S)-np.expand_dims(camera[:,3],axis=1), 
        rcond=None)[0])
    
    #   unit-vectors from camera center to each 3D point
    V = X - np.tile(C, (4, 1))
    V = V / np.linalg.norm(V, ord=2, axis=1, keepdims=True)
    
    # make sure these vectors point forwards from the camera instead of backwards
    V = V*np.expand_dims(np.sign(np.sum(V * np.tile(camera[2,0:3],(4, 1)), axis=1)),axis=1)
    
    #   desired ray segments that are length raysize in these directions 
    V = np.tile(C, (4, 1)) + raysize * V
    
    # append the camera center itself to complete the four tetrahedral vertices
    V=np.vstack([C,V])

    # add camera center to figure
    figobj.add_trace(go.Scatter3d(
        x=[C[0]], 
        y=[C[1]],
        z=[C[2]],
        mode='markers',
        marker=dict(
            size=3,
            color='#ff7f0e'
        )
    )
                    )


    # add tetrahedron to figure
    figobj.add_trace(go.Mesh3d(
        # vertices of tetrahedron
        x=V[:,0],
        y=V[:,1],
        z=V[:,2],

        # i, j and k give the vertices of triangles
        i=[0, 0, 0, 0],
        j=[1, 2, 3, 4],
        k=[2, 3, 4, 1],
        opacity=0.5,
        color='#ff7f0e'
    ))
   
    return 1

In [ ]:
# Convert from Nxm inhomogeneous to Nx(m+1) homogeneous coordinates
def in2hom(X):
    return np.concatenate([X, np.ones((X.shape[0], 1), dtype=np.float32)], axis=1)

# Convert from Nxm homogeneous to Nx(m-1) inhomogeneous coordinates
def hom2in(X):
    return X[:, :-1] / X[:, -1:]

In [ ]:
def computeTwoPlanes(calMatrix, distCoeffs, two_aprilboards_img, april_pkl='./resources_unzipped/AprilBoards.pickle', verify=False):
    """
    Input:

    calMatrix: from Arnav's calibrateCamera
    distCoeffs: from Arnav's calibrateCamera
    two_aprilboards_img : 3-color channeled numpy array img that contains two types of boards
    april_pkl: the april pickle file given in class (applied universally for all the april fine & coarse boards)

    """    
    print("RUN")
    two_aprilboards_img = cv2.cvtColor(two_aprilboards_img, cv2.COLOR_RGB2GRAY)

    #initialize at_detector
    # set up april tag detector (I use default parameters; seems to be OK)

    data = pickle.load(open(april_pkl, 'rb'))
    at_coarseboard = data['at_coarseboard']
    at_fineboard = data['at_fineboard']
    at_detector = Detector(families='tag36h11',
                        nthreads=1,
                        quad_decimate=1.0,
                        quad_sigma=0.0,
                        refine_edges=1,
                        decode_sharpening=0.25,
                        debug=0)

    # detect apriltags and report number of detections. Do this for both boards (fine and coarse).
    imgpoints_fine, objpoints_fine, tagIDs_fine = detect_aprilboard_v2(
        two_aprilboards_img,
        at_fineboard,
        at_detector)
    imgpoints_coarse, objpoints_coarse, tagIDs_coarse = detect_aprilboard_v2(
        two_aprilboards_img,
        at_coarseboard,
        at_detector)

    # compute normalized image coordinates (equivalent to K^{-1}*x)
    # 즉, imgpoints_fine 이라는 이미지 내의 좌표로 표시된 각 QR 코드의 중심부 좌표를 
    #카메라 메트릭스(카메라-to-이미지 principal위치(0,0 이 이미지 중앙, 카메라 렌즈 정중앙에 맞게), focal length)
    #와 굴절율을 고려해서 다시 재조정.
    imgpts_fine_norm = cv2.undistortPoints(imgpoints_fine, calMatrix, distCoeffs)
    imgpts_coarse_norm = cv2.undistortPoints(imgpoints_coarse, calMatrix, distCoeffs)

    # homographies from each board to normalized image pts
    H_fine,_ = cv2.findHomography(objpoints_fine[:,:2],imgpts_fine_norm)
    H_fine = 2*H_fine/(
                    np.linalg.norm(H_fine[:,0]) + np.linalg.norm(H_fine[:,1])
                    )
    # 아래의 np.atleast_2d(np.cross(H_fine[:,0],H_fine[:,1])).T 부분은 R = [r1, r2, r1 x r2] 에서 r1 x r2를 구현한 것이다.
    R_fine = np.hstack((H_fine[:,:2],np.atleast_2d(np.cross(H_fine[:,0],H_fine[:,1])).T))
    # cv2.Rodrigues: https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga61585db663d9da06b68e70cfbf6a1eac
    # Converts a rotation matrix to a rotation vector or vice versa.
    rvec_fine,_ = cv2.Rodrigues(R_fine)
    R_fine,_ = cv2.Rodrigues(rvec_fine)
    tvec_fine=H_fine[:,2]

    H_coarse,_ = cv2.findHomography(objpoints_coarse[:,:2],imgpts_coarse_norm)
    # extract rotation and translation from homography: coarseboard
    H_coarse = 2*H_coarse/(np.linalg.norm(H_coarse[:,0]) + np.linalg.norm(H_coarse[:,1]))
    R_coarse = np.hstack((H_coarse[:,:2],np.atleast_2d(np.cross(H_coarse[:,0],H_coarse[:,1])).T))
    rvec_coarse,_ = cv2.Rodrigues(R_coarse)
    R_coarse,_ = cv2.Rodrigues(rvec_coarse)
    tvec_coarse=H_coarse[:,2]

    
    #LEFT TO DO 1: Get Camera coordinat system 4-vector

    # Apply G^(-T) to Pi_P
    # Pi_c = G^(-T) Pi_P ==> Let Pi_P  be the plane normal (0, 0, 1, 0)

    # We took the Presession 8 quiz and know that  Pi_c can just be calculated (r3, -r3^Tt)
    if verify:
        print("R fine:")
        print(R_fine)

        print("R coarse")
        print(R_coarse)

    # In our case, plane_v is the fine board
    # plane_h is the coarse board

    #Output: plane_h, plane_v -- Two homogeneous 4-vectors corresponding to planes (\Pi_v, \Pi_h) in the camera-centered coordinate system.
    r3_coarse = R_coarse[:,2]
    plane_h = np.hstack([r3_coarse, -r3_coarse.T @ tvec_coarse])

    #plane_v calculation
    r3_fine = R_fine[:,2]
    plane_v = np.hstack([r3_fine, -r3_fine.T @ tvec_fine])
    print("printing plane_h")
    print(plane_h)

    print("printing plane_v")
    print(plane_v)
    #TODO 2: test code for this function

    print(verify)
    if verify:
        plt.imshow(two_aprilboards_img, cmap="gray")
        plt.axis('off')

        #normalized_pH = plane_h / np.linalg.norm(plane_h)
        #normalized_pV = plane_v / np.linalg.norm(plane_v)
        
        pH_3vec = (plane_h / plane_h[3])[:3]
        pV_3vec = (plane_v / plane_v[3])[:3]
        print("pH_3vec:")
        print(pH_3vec)

        plane_intersection = np.cross(pH_3vec, pV_3vec)
        plane_inter_3vec = plane_intersection / plane_intersection[2]
        print("plane_inter_3vec:")
        print(plane_inter_3vec) ## This is normalized point

        # Normalized @
        axes3d = np.float32([[0,0,0], [3,0,0], [0,3,0], [0,0,3]]).reshape(-1,3)
        
        # rotate/translate axes3d object to fineplane origin and project into camera 
        imaxes_fine,_  = cv2.projectPoints(axes3d, rvec_fine, tvec_fine, calMatrix, distCoeffs)

        # eliminate the singleton dimension (quirk of openCV format, a bit annoying)
        imaxes_fine = np.squeeze(imaxes_fine) 

        plt.plot(imaxes_fine[[0,1],0], imaxes_fine[[0,1],1], color='#d62728', linewidth=6)
        plt.plot(imaxes_fine[[0,2],0], imaxes_fine[[0,2],1], color='#2ca02c', linewidth=6)
        plt.plot(imaxes_fine[[0,3],0], imaxes_fine[[0,3],1], color='#1f77b4', linewidth=6)

        # rotate/translate axes3d object to coarseplane origin and project into camera 
        imaxes_coarse,_  = cv2.projectPoints(axes3d, rvec_coarse, tvec_coarse, calMatrix, distCoeffs)

        # eliminate the singleton dimension (quirk of openCV format, a bit annoying)
        imaxes_coarse=np.squeeze(imaxes_coarse)

        plt.plot(imaxes_coarse[[0,1],0], imaxes_coarse[[0,1],1], color='#d62728', linewidth=6)
        plt.plot(imaxes_coarse[[0,2],0], imaxes_coarse[[0,2],1], color='#2ca02c', linewidth=6)
        plt.plot(imaxes_coarse[[0,3],0], imaxes_coarse[[0,3],1], color='#1f77b4', linewidth=6)
        plt.show()
        print("now vis1")
        # Visualization 1: 
        #   Back-project rays through image points (detected tags) and compute
        #   intersection of these rays with the corresponding plane. Plot the
        #   3D intersection points and see if they look right.

        # compute normalized image coordinates (equivalent to K^{-1}*x)
        # Normalized coordinates, when appended with 1 at the end, are the back-projected 
        # ray of point in the camera coordinate system. ( x, y, 1) is the 3d-space point 

        imgpts_fine_norm = cv2.undistortPoints(imgpoints_fine, calMatrix, distCoeffs)
        imgpts_fine_norm = np.squeeze(imgpts_fine_norm)  # remove extraneous size-1 dimension from openCV (annoying)

        imgpts_coarse_norm = cv2.undistortPoints(imgpoints_coarse, calMatrix, distCoeffs)
        imgpts_coarse_norm = np.squeeze(imgpts_coarse_norm)  # remove extraneous size-1 dimension from openCV (annoying)

        # back-projections are homogeneous versions of these 
        # backproj is the backprojected ray to the plane 
        backproj_fine = in2hom(imgpts_fine_norm)
        print(backproj_fine)
        backproj_coarse = in2hom(imgpts_coarse_norm)

        # WORK HERE: 
        # Replace the following two lines to:
        #   1. Compute Nx3 array (intersect_fine) of 3D points that correspond to 
        #      intersections of backproj_fine with plane_fine
        #   2. Compute Nx3 array (intersect_coarse) of 3D points that correspond to 
        #      intersections of backproj_coarse with plane_coarse

        intersect_fine_lambda = -plane_v[3] / np.dot(backproj_fine, plane_v[:3])
        intersect_fine = np.expand_dims(intersect_fine_lambda, axis=1) * backproj_fine

        intersect_coarse_lambda = -plane_h[3] / np.dot(backproj_coarse, plane_h[:3])
        intersect_coarse = np.expand_dims(intersect_coarse_lambda, axis=1) * backproj_coarse

        # WORK HERE: 
        # Replace the following line to properly define the 3x4 camera matrix P based on 
        # the 3x3 matrix calMatrix (for input to the add_camera() function)
        P = np.hstack((calMatrix, np.zeros((3,1))))
        # K = 3x3 , P = 3x4.( Since everything is with respect to the camera coordinate system),

        # P = K[I|0]  ==> So, no rotation 

        # create figure
        print("creating figure!")
        fig = go.Figure()

        # add 3D box corners (as done above)
        fig.add_trace(go.Scatter3d(x=intersect_fine[:,0], 
                                y=intersect_fine[:,1],
                                z=intersect_fine[:,2],
                                mode='markers',
                                marker=dict(
                                    size=2,
                                    color='#1f77b4'
                                )
                                ))

        # add 3D box corners (as done above)
        fig.add_trace(go.Scatter3d(x=intersect_coarse[:,0], 
                                y=intersect_coarse[:,1],
                                z=intersect_coarse[:,2],
                                mode='markers',
                                marker=dict(
                                    size=2,
                                    color='#2ca02c'
                                )
                                ))


        # add camera
        h,w=two_aprilboards_img.shape
        add_camera(h,w,P,1,fig)

        # adjust aspect ratio and initial viewing direction
        fig.update_layout(scene_aspectmode='manual',
                        scene_aspectratio=dict(x=1, y=2, z=3),
                        showlegend=False,
                        scene_camera=dict(
                            up=dict(x=0, y=-1, z=0),
                            center=dict(x=0, y=0, z=0),
                            eye=dict(x=-1, y=-1, z=-5)
                        )
                        )

        fig.show()


        

    return plane_h, plane_v 





In [ ]:
def draw_twoPlanes():
    print(verify)
    if verify:
        plt.imshow(two_aprilboards_img, cmap="gray")
        plt.axis('off')

        #normalized_pH = plane_h / np.linalg.norm(plane_h)
        #normalized_pV = plane_v / np.linalg.norm(plane_v)
        
        pH_3vec = (plane_h / plane_h[3])[:3]
        pV_3vec = (plane_v / plane_v[3])[:3]
        print("pH_3vec:")
        print(pH_3vec)

        plane_intersection = np.cross(pH_3vec, pV_3vec)
        plane_inter_3vec = plane_intersection / plane_intersection[2]
        print("plane_inter_3vec:")
        print(plane_inter_3vec) ## This is normalized point

        # Normalized @
        axes3d = np.float32([[0,0,0], [3,0,0], [0,3,0], [0,0,3]]).reshape(-1,3)
        
        # rotate/translate axes3d object to fineplane origin and project into camera 
        imaxes_fine,_  = cv2.projectPoints(axes3d, rvec_fine, tvec_fine, calMatrix, distCoeffs)

        # eliminate the singleton dimension (quirk of openCV format, a bit annoying)
        imaxes_fine = np.squeeze(imaxes_fine) 

        plt.plot(imaxes_fine[[0,1],0], imaxes_fine[[0,1],1], color='#d62728', linewidth=6)
        plt.plot(imaxes_fine[[0,2],0], imaxes_fine[[0,2],1], color='#2ca02c', linewidth=6)
        plt.plot(imaxes_fine[[0,3],0], imaxes_fine[[0,3],1], color='#1f77b4', linewidth=6)

        # rotate/translate axes3d object to coarseplane origin and project into camera 
        imaxes_coarse,_  = cv2.projectPoints(axes3d, rvec_coarse, tvec_coarse, calMatrix, distCoeffs)

        # eliminate the singleton dimension (quirk of openCV format, a bit annoying)
        imaxes_coarse=np.squeeze(imaxes_coarse)

        plt.plot(imaxes_coarse[[0,1],0], imaxes_coarse[[0,1],1], color='#d62728', linewidth=6)
        plt.plot(imaxes_coarse[[0,2],0], imaxes_coarse[[0,2],1], color='#2ca02c', linewidth=6)
        plt.plot(imaxes_coarse[[0,3],0], imaxes_coarse[[0,3],1], color='#1f77b4', linewidth=6)
        plt.show()
        print("now vis1")
        # Visualization 1: 
        #   Back-project rays through image points (detected tags) and compute
        #   intersection of these rays with the corresponding plane. Plot the
        #   3D intersection points and see if they look right.

        # compute normalized image coordinates (equivalent to K^{-1}*x)
        # Normalized coordinates, when appended with 1 at the end, are the back-projected 
        # ray of point in the camera coordinate system. ( x, y, 1) is the 3d-space point 

        imgpts_fine_norm = cv2.undistortPoints(imgpoints_fine, calMatrix, distCoeffs)
        imgpts_fine_norm = np.squeeze(imgpts_fine_norm)  # remove extraneous size-1 dimension from openCV (annoying)

        imgpts_coarse_norm = cv2.undistortPoints(imgpoints_coarse, calMatrix, distCoeffs)
        imgpts_coarse_norm = np.squeeze(imgpts_coarse_norm)  # remove extraneous size-1 dimension from openCV (annoying)

        # back-projections are homogeneous versions of these 
        # backproj is the backprojected ray to the plane 
        backproj_fine = in2hom(imgpts_fine_norm)
        print(backproj_fine)
        backproj_coarse = in2hom(imgpts_coarse_norm)

        # WORK HERE: 
        # Replace the following two lines to:
        #   1. Compute Nx3 array (intersect_fine) of 3D points that correspond to 
        #      intersections of backproj_fine with plane_fine
        #   2. Compute Nx3 array (intersect_coarse) of 3D points that correspond to 
        #      intersections of backproj_coarse with plane_coarse

        intersect_fine_lambda = -plane_v[3] / np.dot(backproj_fine, plane_v[:3])
        intersect_fine = np.expand_dims(intersect_fine_lambda, axis=1) * backproj_fine

        intersect_coarse_lambda = -plane_h[3] / np.dot(backproj_coarse, plane_h[:3])
        intersect_coarse = np.expand_dims(intersect_coarse_lambda, axis=1) * backproj_coarse

        # WORK HERE: 
        # Replace the following line to properly define the 3x4 camera matrix P based on 
        # the 3x3 matrix calMatrix (for input to the add_camera() function)
        P = np.hstack((calMatrix, np.zeros((3,1))))
        # K = 3x3 , P = 3x4.( Since everything is with respect to the camera coordinate system),

        # P = K[I|0]  ==> So, no rotation 

        # create figure
        print("creating figure!")
        fig = go.Figure()

        # add 3D box corners (as done above)
        fig.add_trace(go.Scatter3d(x=intersect_fine[:,0], 
                                y=intersect_fine[:,1],
                                z=intersect_fine[:,2],
                                mode='markers',
                                marker=dict(
                                    size=2,
                                    color='#1f77b4'
                                )
                                ))

        # add 3D box corners (as done above)
        fig.add_trace(go.Scatter3d(x=intersect_coarse[:,0], 
                                y=intersect_coarse[:,1],
                                z=intersect_coarse[:,2],
                                mode='markers',
                                marker=dict(
                                    size=2,
                                    color='#2ca02c'
                                )
                                ))


        # add camera
        h,w=two_aprilboards_img.shape
        add_camera(h,w,P,1,fig)

        # adjust aspect ratio and initial viewing direction
        fig.update_layout(scene_aspectmode='manual',
                        scene_aspectratio=dict(x=1, y=2, z=3),
                        showlegend=False,
                        scene_camera=dict(
                            up=dict(x=0, y=-1, z=0),
                            center=dict(x=0, y=0, z=0),
                            eye=dict(x=-1, y=-1, z=-5)
                        )
                        )

        fig.show()
